In [1]:
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

warnings.simplefilter("ignore")

In [2]:
train = pd.read_csv('assignment_2_train.csv')
test = pd.read_csv('assignment_2_test.csv')

In [3]:
train.head(3)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*train.shape))

train.shape = 180000 rows, 394 cols
test.shape = 180000 rows, 394 cols


In [5]:
target = train['isFraud']
test_target = test['isFraud']

train = train.drop(['isFraud', 'TransactionID'], axis=1)
test = test.drop(['isFraud', 'TransactionID'], axis=1)


**Задание 1:** отобрать только числовые признаки и обучить модель XGBoost с параметром booster = gbtree. Обучать алгоритм до тех пор, пока метрика качества не перестанет улучшаться на валидационной выборке в течение определенного числа итераций (выбрать значение самостоятельно).

In [6]:
numerical_features = train.select_dtypes(include=[np.number])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features = numerical_features.columns.tolist()

count of numeric_features 378


In [7]:
x_train_num, x_valid_num, y_train_num, y_valid_num = train_test_split(
    train[numerical_features], target, train_size=0.8, random_state=1
)

print("x_train.shape = {} rows, {} cols".format(*x_train_num.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_num.shape))

x_train.shape = 144000 rows, 378 cols
x_valid.shape = 36000 rows, 378 cols


In [8]:
dtrain_xgb_nf = xgb.DMatrix(
    data=x_train_num, label=y_train_num
)
dvalid_xgb_nf = xgb.DMatrix(
    data=x_valid_num, label=y_valid_num
)
dtest_xgb_nf = xgb.DMatrix(data=test[numerical_features]
)

In [9]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [10]:
model_xgb_nf = xgb.train(
    params=params,
    dtrain=dtrain_xgb_nf,
    num_boost_round=500,
    early_stopping_rounds=50,
    evals=[(dtrain_xgb_nf, "train"), (dvalid_xgb_nf, "valid")],
    verbose_eval=50,
    maximize=True,
)

[19:47:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.64988	valid-auc:0.65040
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.88183	valid-auc:0.87420
[100]	train-auc:0.90013	valid-auc:0.88967
[150]	train-auc:0.90824	valid-auc:0.89758
[200]	train-auc:0.91004	valid-auc:0.89892
Stopping. Best iteration:
[167]	train-auc:0.91004	valid-auc:0.89892



In [11]:
pred_xgb_nf = model_xgb_nf.predict(dtest_xgb_nf)

**Задание 2:** обработать категориальные признаки любым способом (который вы знаете) и добавить их к данным. Выполнить задание 1.

In [12]:
categorical_features = train.select_dtypes(include=[np.object])
print(f"Categorical Feature Count {categorical_features.shape[1]}")
categorical_features.head(n=2)

Categorical Feature Count 14


,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN


In [13]:
train_cat = train.copy()
cat_features = train.select_dtypes(include='object').columns.tolist()
cat_features

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [14]:
for cat_colname in cat_features:
    train_cat = pd.concat([train_cat, pd.get_dummies(train_cat[cat_colname], prefix=cat_colname)], axis=1)
    train_cat = train_cat.drop([cat_colname], axis=1)


In [15]:
test_cat = test.copy()

for cat_colname in cat_features:
    test_cat = pd.concat([test_cat, pd.get_dummies(test_cat[cat_colname], prefix=cat_colname)], axis=1)
    test_cat = test_cat.drop([cat_colname], axis=1)

In [16]:
print("train_cat.shape = {} rows, {} cols".format(*train_cat.shape))
print("test_cat.shape = {} rows, {} cols".format(*test_cat.shape))

train_cat.shape = 180000 rows, 529 cols
test_cat.shape = 100001 rows, 525 cols


In [17]:
for colname in train_cat.columns:
    if not (colname in test_cat.columns):
        print(colname)
        train_cat.drop([colname], axis=1, inplace=True)

R_emaildomain_centurylink.net
R_emaildomain_frontiernet.net
R_emaildomain_netzero.net
R_emaildomain_twc.com


In [18]:
print("train_cat.shape = {} rows, {} cols".format(*train_cat.shape))
print("test_cat.shape = {} rows, {} cols".format(*test_cat.shape))

train_cat.shape = 180000 rows, 525 cols
test_cat.shape = 100001 rows, 525 cols


In [19]:
x_train_cat, x_valid_cat, y_train_cat, y_valid_cat = train_test_split(
    train_cat, target, train_size=0.8, random_state=1
)

In [20]:
dtrain_xgb_cf = xgb.DMatrix(np.asmatrix(x_train_cat), label=y_train_cat)
dvalid_xgb_cf = xgb.DMatrix(np.asmatrix(x_valid_cat), label=y_valid_cat)
dtest_xgb_cf = xgb.DMatrix(np.asmatrix(test_cat))

In [21]:
model_xgb_cf = xgb.train(
    params=params,
    dtrain=dtrain_xgb_cf,
    num_boost_round=500,
    early_stopping_rounds=50,
    evals=[(dtrain_xgb_cf, "train"), (dvalid_xgb_cf, "valid")],
    verbose_eval=50,
    maximize=True,
)

[19:49:02] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.64988	valid-auc:0.65040
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.88876	valid-auc:0.88085
[100]	train-auc:0.90947	valid-auc:0.89821
[150]	train-auc:0.91530	valid-auc:0.90275
Stopping. Best iteration:
[139]	train-auc:0.91530	valid-auc:0.90275



In [22]:
pred_xgb_cf = model_xgb_nf.predict(dtest_xgb_cf)

ValueError: feature_names mismatch: ['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', '

In [23]:
set(x_train_cat.columns) - set(test_cat.columns)

set()

In [24]:
test_cat.columns == x_train_cat.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [ ]:
fpr, tpr, _ = roc_curve(test_target, pred_xgb_nf)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='test_xgb_num (area = %0.6f)' % roc_auc)

In [ ]:
fpr, tpr, _ = roc_curve(test_target, pred_xgb_cf)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='test_xgb_cf (area = %0.6f)' % roc_auc)